# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

In [1]:
# load missing library
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128        # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True   # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.08s)
creating index...


  0%|          | 729/414113 [00:00<02:01, 3389.98it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:35<00:00, 4349.30it/s]


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

torch.Size([128, 13, 512])
Epoch [1/3], Step [1/3236], Loss: 9.1913, Perplexity: 9811.5791torch.Size([128, 14, 512])
Epoch [1/3], Step [2/3236], Loss: 8.9454, Perplexity: 7672.1749torch.Size([128, 11, 512])
Epoch [1/3], Step [3/3236], Loss: 8.6238, Perplexity: 5562.3314torch.Size([128, 13, 512])
Epoch [1/3], Step [4/3236], Loss: 8.2786, Perplexity: 3938.5589torch.Size([128, 12, 512])
Epoch [1/3], Step [5/3236], Loss: 7.7338, Perplexity: 2284.1808torch.Size([128, 13, 512])
Epoch [1/3], Step [6/3236], Loss: 7.1909, Perplexity: 1327.2850torch.Size([128, 12, 512])
Epoch [1/3], Step [7/3236], Loss: 6.4613, Perplexity: 639.9012torch.Size([128, 14, 512])
Epoch [1/3], Step [8/3236], Loss: 5.7999, Perplexity: 330.2678torch.Size([128, 16, 512])
Epoch [1/3], Step [9/3236], Loss: 5.5487, Perplexity: 256.9158torch.Size([128, 13, 512])
Epoch [1/3], Step [10/3236], Loss: 5.2362, Perplexity: 187.9522torch.Size([128, 16, 512])
Epoch [1/3], Step [11/3236], Loss: 5.0579, Perplexity: 157.2590torch.Size([1

Epoch [1/3], Step [92/3236], Loss: 4.0762, Perplexity: 58.9202torch.Size([128, 11, 512])
Epoch [1/3], Step [93/3236], Loss: 3.6896, Perplexity: 40.0282torch.Size([128, 16, 512])
Epoch [1/3], Step [94/3236], Loss: 3.8374, Perplexity: 46.4065torch.Size([128, 15, 512])
Epoch [1/3], Step [95/3236], Loss: 3.6139, Perplexity: 37.1112torch.Size([128, 14, 512])
Epoch [1/3], Step [96/3236], Loss: 3.4796, Perplexity: 32.4476torch.Size([128, 15, 512])
Epoch [1/3], Step [97/3236], Loss: 3.6134, Perplexity: 37.0929torch.Size([128, 11, 512])
Epoch [1/3], Step [98/3236], Loss: 3.6876, Perplexity: 39.9508torch.Size([128, 13, 512])
Epoch [1/3], Step [99/3236], Loss: 3.4802, Perplexity: 32.4663torch.Size([128, 12, 512])
Epoch [1/3], Step [100/3236], Loss: 3.5885, Perplexity: 36.1797
torch.Size([128, 11, 512])
Epoch [1/3], Step [101/3236], Loss: 3.6012, Perplexity: 36.6410torch.Size([128, 14, 512])
Epoch [1/3], Step [102/3236], Loss: 3.5884, Perplexity: 36.1765torch.Size([128, 14, 512])
Epoch [1/3], Step

Epoch [1/3], Step [183/3236], Loss: 3.2857, Perplexity: 26.7264torch.Size([128, 14, 512])
Epoch [1/3], Step [184/3236], Loss: 3.2121, Perplexity: 24.8310torch.Size([128, 13, 512])
Epoch [1/3], Step [185/3236], Loss: 3.4262, Perplexity: 30.7595torch.Size([128, 13, 512])
Epoch [1/3], Step [186/3236], Loss: 3.0378, Perplexity: 20.8602torch.Size([128, 11, 512])
Epoch [1/3], Step [187/3236], Loss: 3.5327, Perplexity: 34.2158torch.Size([128, 13, 512])
Epoch [1/3], Step [188/3236], Loss: 3.2049, Perplexity: 24.6542torch.Size([128, 15, 512])
Epoch [1/3], Step [189/3236], Loss: 3.2656, Perplexity: 26.1959torch.Size([128, 11, 512])
Epoch [1/3], Step [190/3236], Loss: 3.3843, Perplexity: 29.4964torch.Size([128, 13, 512])
Epoch [1/3], Step [191/3236], Loss: 3.3495, Perplexity: 28.4896torch.Size([128, 16, 512])
Epoch [1/3], Step [192/3236], Loss: 3.5269, Perplexity: 34.0190torch.Size([128, 11, 512])
Epoch [1/3], Step [193/3236], Loss: 3.3577, Perplexity: 28.7242torch.Size([128, 15, 512])
Epoch [1/3

Epoch [1/3], Step [274/3236], Loss: 3.0153, Perplexity: 20.3945torch.Size([128, 11, 512])
Epoch [1/3], Step [275/3236], Loss: 3.4206, Perplexity: 30.5873torch.Size([128, 16, 512])
Epoch [1/3], Step [276/3236], Loss: 3.3391, Perplexity: 28.1942torch.Size([128, 11, 512])
Epoch [1/3], Step [277/3236], Loss: 3.2594, Perplexity: 26.0340torch.Size([128, 12, 512])
Epoch [1/3], Step [278/3236], Loss: 3.1143, Perplexity: 22.5174torch.Size([128, 15, 512])
Epoch [1/3], Step [279/3236], Loss: 3.1786, Perplexity: 24.0134torch.Size([128, 13, 512])
Epoch [1/3], Step [280/3236], Loss: 3.0881, Perplexity: 21.9355torch.Size([128, 14, 512])
Epoch [1/3], Step [281/3236], Loss: 3.2894, Perplexity: 26.8277torch.Size([128, 11, 512])
Epoch [1/3], Step [282/3236], Loss: 3.2780, Perplexity: 26.5221torch.Size([128, 14, 512])
Epoch [1/3], Step [283/3236], Loss: 3.1865, Perplexity: 24.2047torch.Size([128, 11, 512])
Epoch [1/3], Step [284/3236], Loss: 3.2900, Perplexity: 26.8431torch.Size([128, 13, 512])
Epoch [1/3

Epoch [1/3], Step [365/3236], Loss: 3.1959, Perplexity: 24.4316torch.Size([128, 16, 512])
Epoch [1/3], Step [366/3236], Loss: 3.1098, Perplexity: 22.4162torch.Size([128, 15, 512])
Epoch [1/3], Step [367/3236], Loss: 3.0727, Perplexity: 21.6008torch.Size([128, 12, 512])
Epoch [1/3], Step [368/3236], Loss: 3.0551, Perplexity: 21.2229torch.Size([128, 24, 512])
Epoch [1/3], Step [369/3236], Loss: 4.0264, Perplexity: 56.0606torch.Size([128, 11, 512])
Epoch [1/3], Step [370/3236], Loss: 3.0270, Perplexity: 20.6343torch.Size([128, 13, 512])
Epoch [1/3], Step [371/3236], Loss: 2.9113, Perplexity: 18.3805torch.Size([128, 17, 512])
Epoch [1/3], Step [372/3236], Loss: 3.3328, Perplexity: 28.0179torch.Size([128, 11, 512])
Epoch [1/3], Step [373/3236], Loss: 3.1415, Perplexity: 23.1396torch.Size([128, 12, 512])
Epoch [1/3], Step [374/3236], Loss: 2.9923, Perplexity: 19.9319torch.Size([128, 17, 512])
Epoch [1/3], Step [375/3236], Loss: 3.2043, Perplexity: 24.6390torch.Size([128, 11, 512])
Epoch [1/3

...

Epoch [2/3], Step [760/3236], Loss: 2.0431, Perplexity: 7.7143torch.Size([128, 13, 512])
Epoch [2/3], Step [761/3236], Loss: 2.0201, Perplexity: 7.5388torch.Size([128, 13, 512])
Epoch [2/3], Step [762/3236], Loss: 2.1238, Perplexity: 8.3626torch.Size([128, 11, 512])
Epoch [2/3], Step [763/3236], Loss: 2.1494, Perplexity: 8.5798torch.Size([128, 12, 512])
Epoch [2/3], Step [764/3236], Loss: 2.0724, Perplexity: 7.9439torch.Size([128, 14, 512])
Epoch [2/3], Step [765/3236], Loss: 2.1155, Perplexity: 8.2936torch.Size([128, 11, 512])
Epoch [2/3], Step [766/3236], Loss: 2.2257, Perplexity: 9.2598torch.Size([128, 13, 512])
Epoch [2/3], Step [767/3236], Loss: 2.1962, Perplexity: 8.9908torch.Size([128, 13, 512])
Epoch [2/3], Step [768/3236], Loss: 2.0710, Perplexity: 7.9330torch.Size([128, 13, 512])
Epoch [2/3], Step [769/3236], Loss: 2.1152, Perplexity: 8.2916torch.Size([128, 13, 512])
Epoch [2/3], Step [770/3236], Loss: 2.0500, Perplexity: 7.7679torch.Size([128, 11, 512])
Epoch [2/3], Step [77

Epoch [2/3], Step [852/3236], Loss: 1.9473, Perplexity: 7.0095torch.Size([128, 16, 512])
Epoch [2/3], Step [853/3236], Loss: 2.1334, Perplexity: 8.4434torch.Size([128, 14, 512])
Epoch [2/3], Step [854/3236], Loss: 2.1761, Perplexity: 8.8120torch.Size([128, 13, 512])
Epoch [2/3], Step [855/3236], Loss: 2.0015, Perplexity: 7.4005torch.Size([128, 20, 512])
Epoch [2/3], Step [856/3236], Loss: 2.7673, Perplexity: 15.9152torch.Size([128, 14, 512])
Epoch [2/3], Step [857/3236], Loss: 2.1141, Perplexity: 8.2820torch.Size([128, 13, 512])
Epoch [2/3], Step [858/3236], Loss: 2.0259, Perplexity: 7.5829torch.Size([128, 14, 512])
Epoch [2/3], Step [859/3236], Loss: 2.0756, Perplexity: 7.9696torch.Size([128, 12, 512])
Epoch [2/3], Step [860/3236], Loss: 2.1539, Perplexity: 8.6181torch.Size([128, 10, 512])
Epoch [2/3], Step [861/3236], Loss: 2.2872, Perplexity: 9.8473torch.Size([128, 16, 512])
Epoch [2/3], Step [862/3236], Loss: 2.3451, Perplexity: 10.4348torch.Size([128, 10, 512])
Epoch [2/3], Step [

Epoch [2/3], Step [944/3236], Loss: 2.0974, Perplexity: 8.1452torch.Size([128, 16, 512])
Epoch [2/3], Step [945/3236], Loss: 2.3528, Perplexity: 10.5148torch.Size([128, 13, 512])
Epoch [2/3], Step [946/3236], Loss: 2.0770, Perplexity: 7.9802torch.Size([128, 12, 512])
Epoch [2/3], Step [947/3236], Loss: 2.0374, Perplexity: 7.6707torch.Size([128, 14, 512])
Epoch [2/3], Step [948/3236], Loss: 2.0606, Perplexity: 7.8508torch.Size([128, 13, 512])
Epoch [2/3], Step [949/3236], Loss: 1.9727, Perplexity: 7.1903torch.Size([128, 16, 512])
Epoch [2/3], Step [950/3236], Loss: 2.3153, Perplexity: 10.1276torch.Size([128, 12, 512])
Epoch [2/3], Step [951/3236], Loss: 2.0524, Perplexity: 7.7865torch.Size([128, 12, 512])
Epoch [2/3], Step [952/3236], Loss: 1.9207, Perplexity: 6.8259torch.Size([128, 13, 512])
Epoch [2/3], Step [953/3236], Loss: 2.0549, Perplexity: 7.8058torch.Size([128, 15, 512])
Epoch [2/3], Step [954/3236], Loss: 2.2291, Perplexity: 9.2918torch.Size([128, 14, 512])
Epoch [2/3], Step [

Epoch [2/3], Step [1035/3236], Loss: 2.2200, Perplexity: 9.2077torch.Size([128, 16, 512])
Epoch [2/3], Step [1036/3236], Loss: 2.1948, Perplexity: 8.9781torch.Size([128, 13, 512])
Epoch [2/3], Step [1037/3236], Loss: 2.0312, Perplexity: 7.6235torch.Size([128, 12, 512])
Epoch [2/3], Step [1038/3236], Loss: 1.9873, Perplexity: 7.2960torch.Size([128, 11, 512])
Epoch [2/3], Step [1039/3236], Loss: 2.0706, Perplexity: 7.9299torch.Size([128, 15, 512])
Epoch [2/3], Step [1040/3236], Loss: 2.1110, Perplexity: 8.2567torch.Size([128, 12, 512])
Epoch [2/3], Step [1041/3236], Loss: 2.0176, Perplexity: 7.5204torch.Size([128, 14, 512])
Epoch [2/3], Step [1042/3236], Loss: 2.0448, Perplexity: 7.7273torch.Size([128, 12, 512])
Epoch [2/3], Step [1043/3236], Loss: 2.1091, Perplexity: 8.2407torch.Size([128, 13, 512])
Epoch [2/3], Step [1044/3236], Loss: 2.0968, Perplexity: 8.1401torch.Size([128, 13, 512])
Epoch [2/3], Step [1045/3236], Loss: 2.0949, Perplexity: 8.1248torch.Size([128, 12, 512])
Epoch [2/3

Epoch [2/3], Step [1126/3236], Loss: 2.0252, Perplexity: 7.5776torch.Size([128, 11, 512])
Epoch [2/3], Step [1127/3236], Loss: 2.1107, Perplexity: 8.2537torch.Size([128, 14, 512])
Epoch [2/3], Step [1128/3236], Loss: 2.0706, Perplexity: 7.9300

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.